In [1]:
import numpy as np
import pandas as pd
import sys

from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import train_test_split


from lightgbm import LGBMClassifier

In [2]:
df_train = np.load('../cache/train_p_stage1_fe2.npy')
df_test = np.load('../cache/test_p_stage1_fe2.npy')
df = pd.read_csv('../cache/stage1_labels.csv')
y = df['y'].values

df1 = pd.read_csv('../cache/stage1_p_test_labels.csv')
test_labels = df1['y'].values

wts_per_class = np.load('../cache/stage1_train_weights_per_class.npy')
wts_per_class = wts_per_class.tolist()
print(wts_per_class)

{1: 4.846830985915493, 2: 6.347345132743363, 3: 36.31460674157304, 4: 3.8411078717201166, 5: 12.723140495867769, 6: 11.076363636363636, 7: 2.484784889821616, 8: 173.78947368421052, 9: 88.75675675675676}


In [3]:
w = np.array([wts_per_class[j] for j in y],)

In [6]:
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'metric': 'multi_logloss',
#     'num_leaves': 50,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'colsample_bytree': 0.8,
#     'n_estimators': 500,
#     'verbose': 1
# }

params = {'num_leaves':[150,200], 'max_depth':[7,10],
          'learning_rate':[0.05,0.1],'max_bin':[100,200], 'n_estimators': [200]}
# x1, x2, y1, y2 = train_test_split(df_train, y, test_size=0.2, random_state=42)

lgbm = LGBMClassifier(objective='multiclass', sample_weight=w)



clf = GridSearchCV(lgbm, params, n_jobs=4, 
                   cv=5,
                   scoring = 'neg_log_loss',
                   verbose=2, refit=True)



In [7]:
clf.fit(df_train, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV]  learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150, total=55.8min
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV]  learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150, total=55.9min
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150, total=56.0min
[CV] learning_rate=0.05, max_bin=100, max_depth=7, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=100, max_d

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 505.9min


[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=150, total=49.3min
[CV] learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total=46.5min
[CV] learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=150, total=47.2min
[CV] learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total=44.8min
[CV] learning_rate=0.1, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV]  learning_rate=0.05, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total=46.4min
[CV] learning_rate=0.1, max_bin=100, max_depth=7, n_estimators=200, num_leaves=150 
[CV]  learning_rate=0.1, max_bin=100, max_depth=7, n_estimators=200, 

[CV]  learning_rate=0.1, max_bin=200, max_depth=10, n_estimators=200, num_leaves=200, total= 8.5min


[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed: 844.5min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0.0, n_estimators=10, n_jobs=-1,
        num_leaves=31, objective='multiclass', random_state=0,
        reg_alpha=0.0, reg_lambda=0.0,
        sample_weight=array([ 4.84683,  6.34735, ...,  3.84111,  3.84111]),
        silent=True, subsample=1.0, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'num_leaves': [150, 200], 'max_depth': [7, 10], 'learning_rate': [0.05, 0.1], 'max_bin': [100, 200], 'n_estimators': [200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=2)

In [8]:
print(clf.best_score_)
print(clf.best_params_)

# -1.87263813814
# {'learning_rate': 0.05, 'max_bin': 100, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 150}
#final multi_log_loss: 1.2407789319265683
# final f1_score: 0.5135869565217391
# final accuracy: 0.5135869565217391
# [[53  1  1 24 11  0  4  0  0]
#  [ 3 11  0  1  0  0 31  0  0]
#  [ 1  0  1  2  0  0  3  0  0]
#  [12  2  0 38  0  2 11  0  0]
#  [ 6  0  0  6  7  5  1  0  0]
#  [ 4  2  0  2  7  1  6  0  0]
#  [ 1 13  1  7  4  0 75  0  0]
#  [ 1  0  0  0  0  0  1  0  0]
#  [ 1  0  0  1  0  0  1  0  3]]

-1.87263813814
{'learning_rate': 0.05, 'max_bin': 100, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 150}


In [9]:
test_preds = clf.predict_proba(df_test)
print(test_preds)

[[ 0.66021328  0.03584617  0.00529511 ...,  0.01430333  0.0039112
   0.0025975 ]
 [ 0.10766885  0.13915331  0.07436388 ...,  0.27665764  0.00477525
   0.01110223]
 [ 0.03276992  0.10890619  0.00896667 ...,  0.63635537  0.00145999
   0.00139406]
 ..., 
 [ 0.03803813  0.10224106  0.00838041 ...,  0.5486421   0.00534403
   0.00181727]
 [ 0.16437162  0.17153948  0.0402269  ...,  0.08870735  0.00372967
   0.01362961]
 [ 0.69055869  0.07781394  0.00404339 ...,  0.01235439  0.00156096
   0.00322503]]


In [10]:
score2 = log_loss(test_labels, test_preds, labels = list(range(1,10)))
print('final multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10)), average='micro')
print('final f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, test_preds.argmax(axis=1)+1)
print('final accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10))))

final multi_log_loss: 1.2407789319265683
final f1_score: 0.5135869565217391
final accuracy: 0.5135869565217391
[[53  1  1 24 11  0  4  0  0]
 [ 3 11  0  1  0  0 31  0  0]
 [ 1  0  1  2  0  0  3  0  0]
 [12  2  0 38  0  2 11  0  0]
 [ 6  0  0  6  7  5  1  0  0]
 [ 4  2  0  2  7  1  6  0  0]
 [ 1 13  1  7  4  0 75  0  0]
 [ 1  0  0  0  0  0  1  0  0]
 [ 1  0  0  1  0  0  1  0  3]]


In [ ]:
x1,x2,y1,y2 = train_test_split(df_train, y, test_size=0.2, random_state=42)

In [16]:
w1 = np.array([wts_per_class[j] for j in y1], )
w2 = np.array([wts_per_class[j] for j in y2], )

In [49]:
clf1 = LGBMClassifier(objective='multiclass', learning_rate= 0.03, 
                     max_bin= 100, max_depth= 7, n_estimators= 200, num_leaves= 150)

In [50]:
clf1.fit(x1, y1, sample_weight=w1, eval_set=[(x2, y2)], eval_sample_weight=[w2], 
        eval_metric='multi_logloss', early_stopping_rounds=100)

[1]	valid_0's multi_logloss: 2.14339
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 2.09826
[3]	valid_0's multi_logloss: 2.05681
[4]	valid_0's multi_logloss: 2.0193
[5]	valid_0's multi_logloss: 1.98246
[6]	valid_0's multi_logloss: 1.95
[7]	valid_0's multi_logloss: 1.9186
[8]	valid_0's multi_logloss: 1.88975
[9]	valid_0's multi_logloss: 1.86215
[10]	valid_0's multi_logloss: 1.83632
[11]	valid_0's multi_logloss: 1.81117
[12]	valid_0's multi_logloss: 1.78832
[13]	valid_0's multi_logloss: 1.76499
[14]	valid_0's multi_logloss: 1.74225
[15]	valid_0's multi_logloss: 1.72083
[16]	valid_0's multi_logloss: 1.69969
[17]	valid_0's multi_logloss: 1.68105
[18]	valid_0's multi_logloss: 1.66281
[19]	valid_0's multi_logloss: 1.64521
[20]	valid_0's multi_logloss: 1.62857
[21]	valid_0's multi_logloss: 1.61304
[22]	valid_0's multi_logloss: 1.59838
[23]	valid_0's multi_logloss: 1.58474
[24]	valid_0's multi_logloss: 1.5716
[25]	valid_0's multi_logloss: 1.55786
[2

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.03,
        max_bin=100, max_depth=7, min_child_samples=10, min_child_weight=5,
        min_split_gain=0.0, n_estimators=200, n_jobs=-1, num_leaves=150,
        objective='multiclass', random_state=0, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1)

In [51]:
# Notes -
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'metric': 'multi_logloss',
#     'num_leaves': 50,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'colsample_bytree': 0.8,
#     'n_estimators': 500,
#     'verbose': 1
# }
# with above params we get
# final multi_log_loss: 1.3155948686089638
# final f1_score: 0.0517503805175038
# final accuracy: 0.04619565217391304
# [[ 0  1 26  7  4  6  0  0  0]
#  [ 6  0  2  0  0 32  0  0  0]
#  [ 0  1  2  0  0  3  0  0  0]
#  [ 4  0 42  0  0  6  0  0  0]
#  [ 0  0  7  1  8  3  0  0  0]
#  [ 2  0  2  4  4  7  0  0  0]
#  [11  4  4  1  1 80  0  0  0]
#  [ 1  0  0  0  0  1  0  0  0]
#  [ 0  0  2  0  0  1  0  3  0]]

# params = {'num_leaves':150, 'objective':'multiclass','max_depth':7,'metric': 'multi_logloss',
#           'learning_rate':.05,'max_bin':200, 'n_estimators': 500}
# x1, x2, y1, y2 = train_test_split(df_train, y, test_size=0.2, random_state=42)
# final multi_log_loss: 1.5149298479876543
# final f1_score: 0.07496251874062969
# final accuracy: 0.06793478260869565
# [[10  2 17  4 13  5  0  0  0]
#  [12  0  4  3  1 22  0  0  0]
#  [ 0  1  1  1  2  2  0  0  0]
#  [ 4  5 37  0  2  4  0  0  0]
#  [ 0  2  5  7  8  1  0  0  0]
#  [ 5  0  1  2  7  4  0  1  0]
#  [25  5  5  4  5 52  2  0  0]
#  [ 1  0  0  0  0  1  0  0  0]
#  [ 0  1  1  1  0  0  0  1  0]]

In [52]:
test_preds = clf1.predict_proba(df_test)
print(test_preds)

[[ 0.53792035  0.01435237  0.01592424 ...,  0.07223032  0.00724894
   0.00713548]
 [ 0.21290062  0.06401941  0.07136421 ...,  0.24289616  0.02149987
   0.02172991]
 [ 0.04409278  0.08274295  0.00949478 ...,  0.67344784  0.00791152
   0.00790701]
 ..., 
 [ 0.06116101  0.05060196  0.02529718 ...,  0.58320259  0.01149006
   0.01206347]
 [ 0.25471644  0.14795098  0.07902569 ...,  0.08055062  0.01407895
   0.01256965]
 [ 0.42351744  0.06911253  0.02646092 ...,  0.09477255  0.00974634
   0.00968133]]


In [53]:
score2 = log_loss(test_labels, test_preds, labels = list(range(1,10)))
print('final multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10)), average='micro')
print('final f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, test_preds.argmax(axis=1)+1)
print('final accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10))))

# final multi_log_loss: 1.5149298479876543
# final f1_score: 0.07496251874062969
# final accuracy: 0.06793478260869565
# [[10  2 17  4 13  5  0  0  0]
#  [12  0  4  3  1 22  0  0  0]
#  [ 0  1  1  1  2  2  0  0  0]
#  [ 4  5 37  0  2  4  0  0  0]
#  [ 0  2  5  7  8  1  0  0  0]
#  [ 5  0  1  2  7  4  0  1  0]
#  [25  5  5  4  5 52  2  0  0]
#  [ 1  0  0  0  0  1  0  0  0]
#  [ 0  1  1  1  0  0  0  1  0]]

final multi_log_loss: 1.4125463050940268
final f1_score: 0.47554347826086957
final accuracy: 0.47554347826086957
[[51  3  3 12  9 10  6  0  0]
 [ 8 12  0  2  1  1 22  0  0]
 [ 1  0  2  1  1  1  1  0  0]
 [16  2  2 35  1  2  7  0  0]
 [ 2  0  2  6  6  7  2  0  0]
 [ 2  3  0  2  2  7  6  0  0]
 [ 4 22  0  7  5  2 61  0  0]
 [ 0  1  0  0  0  0  1  0  0]
 [ 0  0  0  3  1  0  1  0  1]]
